In [1]:
import numpy as np, random, operator, pandas as pd

import networkx as nx

In [2]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"
    
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [3]:
    
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations,rewiring):
    pop = initialPopulation(popSize, population)
    #initial watts and strogatz network
    G = nx.generators.random_graphs.watts_strogatz_graph(popSize,3,rewiring)
    
    scores = []    
    
    for l in range(0,generations):            
        gen_scores=[]    
        for j in range(0, popSize):        
            candidato = pop[j]            
            vecinos=[]
            for k in G[j]:                
                vecinos.append(k)
                
            if(len(vecinos)>0):            
                #Select best neighbour
                mejor_vecino = 0
                puntaje_mejor_vecino = 100000000
                
                for k in G[j]:                    
                    fitness_vecino = 1/Fitness(pop[k]).routeFitness()                    
                    if(fitness_vecino<puntaje_mejor_vecino):
                        puntaje_mejor_vecino = fitness_vecino
                        mejor_vecino = k
                                    
                if(puntaje_mejor_vecino<100000000):            
                    pareja = pop[mejor_vecino]                                                                                        
                    child = breed(candidato, pareja)
                    child = mutate(child,mutationRate)
                    
                    score_individuo  = 1/Fitness(candidato).routeFitness()
                    score_child = 1/Fitness(child).routeFitness()                                        
                    
                    if(score_child < score_individuo):                        
                        pop[j] =  child                        
                        G.add_edge(j,mejor_vecino)
                        gen_scores.append(score_child)
                    else:
                        gen_scores.append(score_individuo)
    
        for j in range(0, popSize):            
            aristas = G.adj[j]
            
            vecinos=[]
            for k in aristas:                
                vecinos.append(k)
            
            for i in range(0,len(vecinos)):
                if random.random() < rewiring:                    
                    random_index = int(random.random()*100)                    
                    G.remove_edge(j, vecinos[i])                    
                    G.add_edge(j,random_index)                    
        
        gen_best = min(gen_scores)        
        scores.append(gen_best)
        
    return scores

In [4]:

cityList = []

#25 cities
for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

iteraciones = []      
promedios = []
#Number of iterations per run
maxiter = 50
#Number of experiments, to then gained the average from them in each iteration
numero_iteraciones = 50

for i in range(0,numero_iteraciones):
    print(i)
    #popsize 100, elite search 20%, rewiring 30%, mutation rate 10%
    iteracionTemporal = geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=maxiter,rewiring = 0.3)
    iteraciones.append(iteracionTemporal)
    
for i in range(0,maxiter):
    sumatoria_temporal = 0
    for j in range(0,numero_iteraciones):        
        sumatoria_temporal = sumatoria_temporal + iteraciones[j][i]
            
    promedio_temporal = sumatoria_temporal / numero_iteraciones
    promedios.append(promedio_temporal)
    
print("\n")
print("Print averages per iteration:")
for i in range(0,maxiter):    
    print("%.9f"%promedios[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


Print averages per iteration:
2070.391841070
2006.342843235
1940.315586396
1886.633270127
1844.380303772
1798.136334755
1758.673776562
1707.183031266
1664.246921545
1627.708833426
1591.477853080
1567.392694510
1539.885769036
1527.547950517
1507.006717838
1489.011433763
1460.987977912
1437.042381915
1411.714397866
1401.800012866
1392.539443083
1375.144549489
1359.326982804
1335.598551675
1324.550114213
1308.146150006
1297.053406855
1281.353962228
1272.107591240
1267.635658440
1252.425530333
1247.525099891
1231.636851307
1219.990568761
1208.848675473
1204.703975500
1198.881320665
1193.851837429
1184.341755274
1177.384580870
1169.654908386
1163.782247680
1158.710563456
1149.221666758
1144.886384306
1145.605690060
1141.154611891
1135.249167902
1130.252149782
1129.449499887
